# Spatio-Temporal Prediction of Parking Violation Ratios

**Official Submission Notebook**

This notebook presents the final machine learning pipeline to predict the `invalid_ratio` for specific parking zones and times.
It includes three main sections:
1. **Improved Model:** CatBoost with One-Hot Encoding.
2. **Optimized CatBoost:** Utilizing Native Categorical Support for superior performance.
3. **Advanced Neural Network:** Featuring Spatial Embeddings via K-Means Clustering.

 ## 1. Improved Model: CatBoost (One-Hot Encoding)

This section demonstrates a basic CatBoost approach using traditional one-hot encoding for categorical variables.

In [ ]:
# Installation (required once in Colab)
!pip install catboost

# Import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import spearmanr
import gc # Helps with memory management
from catboost import CatBoostRegressor

print("Installation and imports completed!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 7.5 MB/s eta 0:00:00
Installation und Importe fertig!


In [ ]:
print("Loading training data...")
x_train_path = '/content/x_train_final_asAbTs5.csv'
y_train_path = '/content/y_train_final_YYyFil7.csv'

# Load data
X_raw = pd.read_csv(x_train_path)
y_raw = pd.read_csv(y_train_path)

# Merge on the ID column
id_col = 'Column1' if 'Column1' in X_raw.columns else X_raw.columns[0]
data = pd.merge(X_raw, y_raw, on=id_col, how='inner')

# Separate target variable and features
target_col = 'invalid_ratio'
y = data[target_col]
X = data.drop([target_col, id_col], axis=1)

# Clean up memory (delete old variables)
del X_raw, y_raw
gc.collect()

print(f"Data loaded. Number of rows: {len(X)}")

Lade Trainingsdaten...
Daten geladen. Anzahl Zeilen: 6076546


In [ ]:
print("Preparing features...")

# Convert categorical columns (One-Hot Encoding)
categorical_cols = ['day_of_week', 'month_of_year', 'hour']
X = pd.get_dummies(X, columns=categorical_cols, dummy_na=False)
X = X.fillna(0)

# Split: 80% Training, 20% Validation (internal test set)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training Shape: {X_train.shape}")
print(f"Validation Shape: {X_val.shape}")

Bereite Features vor...
Training Shape: (4861236, 39)
Validation Shape: (1215310, 39)


In [ ]:
print("Starting CatBoost Training...")

model = CatBoostRegressor(
    iterations=500,        # Number of trees (500 is a good starting point)
    learning_rate=0.1,     # Learning rate
    depth=6,               # Depth of the trees (6 is standard)
    loss_function='RMSE',  # Optimize for Root Mean Squared Error
    random_seed=42,
    verbose=100,           # Show progress every 100 steps
    allow_writing_files=False # Prevents writing intermediate files
)

# Start training (with validation data for early stopping)
model.fit(X_train, y_train, eval_set=(X_val, y_val), early_stopping_rounds=50)

print("Training completed!")

Starte CatBoost Training...
0:	learn: 0.3639116	test: 0.3641546	best: 0.3641546 (0)	total: 1.26s	remaining: 10m 27s
100:	learn: 0.3335251	test: 0.3335528	best: 0.3335528 (100)	total: 1m 16s	remaining: 5m 2s
200:	learn: 0.3290232	test: 0.3290597	best: 0.3290597 (200)	total: 2m 36s	remaining: 3m 52s
300:	learn: 0.3258046	test: 0.3258651	best: 0.3258651 (300)	total: 3m 51s	remaining: 2m 32s
400:	learn: 0.3229678	test: 0.3230352	best: 0.3230352 (400)	total: 5m 8s	remaining: 1m 16s
499:	learn: 0.3209734	test: 0.3210646	best: 0.3210646 (499)	total: 6m 24s	remaining: 0us

bestTest = 0.3210646381
bestIteration = 499

Training abgeschlossen!


In [ ]:
# Predict on validation data
y_pred = model.predict(X_val)

# Calculate metrics
spearman_score, _ = spearmanr(y_val, y_pred)
mse = mean_squared_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)

print("="*40)
print(f"🏆 Spearman Correlation: {spearman_score:.5f}")
print(f"📉 Mean Squared Error:   {mse:.5f}")
print(f"📊 R² Score:             {r2:.5f}")
print("="*40)

🏆 Spearman Correlation: 0.46093
📉 Mean Squared Error:   0.10308
📊 R² Score:             0.24118


In [ ]:
# --- CREATE SUBMISSION ---
print("Preparing final submission...")

# 1. Retrain on the FULL dataset (Training + Validation)
model.fit(X, y, verbose=100)

# 2. Store column names (important for alignment!)
model_columns = X.columns.tolist()

# 3. IMPORTANT: Free up memory! Delete training data.
del X, y, X_train, X_val, y_train, y_val
gc.collect()
print("Memory cleared.")

# 4. Testdaten laden
print("Loading test data...")
X_test_real = pd.read_csv('/content/x_test_final_fIrnA7Q.csv')

# 5. Prepare test data (same steps as above)
if id_col in X_test_real.columns:
    X_test_real = X_test_real.drop(id_col, axis=1)
if target_col in X_test_real.columns:
    X_test_real = X_test_real.drop(target_col, axis=1)

# One-Hot Encoding for test data
X_test_real = pd.get_dummies(X_test_real, columns=categorical_cols, dummy_na=False)

# 6. Align columns (Prevents crashing by matching train columns)
print("Aligning columns...")
# Force test data into exactly the same shape as training data
X_test_ready = X_test_real.reindex(columns=model_columns, fill_value=0)

# Delete raw data
del X_test_real
gc.collect()

# 7. Predict & Save
print("Generating predictions...")
final_predictions = model.predict(X_test_ready)

submission = pd.DataFrame(final_predictions, columns=['invalid_ratio'])
submission.index = np.arange(len(submission))
submission.index.name = None
output_filename = 'submission_catboost.csv'
submission.to_csv(output_filename, index=True)

print(f"Done! File '{output_filename}' saved.")

# Start download
from google.colab import files
try:
    files.download(output_filename)
except:
    print("Please start the download manually if it fails.")

Bereite finale Abgabe vor...
0:	learn: 0.3639268	total: 1.87s	remaining: 15m 32s
100:	learn: 0.3332953	total: 1m 34s	remaining: 6m 12s
200:	learn: 0.3287952	total: 3m 3s	remaining: 4m 33s
300:	learn: 0.3254737	total: 4m 34s	remaining: 3m 1s
400:	learn: 0.3230885	total: 6m 6s	remaining: 1m 30s
499:	learn: 0.3209508	total: 7m 36s	remaining: 0us
Speicher bereinigt.
Lade Testdaten (x_test.csv)...
Gleiche Spalten an...
Erstelle Vorhersage...
Fertig! Datei 'submission_catboost.csv' gespeichert.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 2. Optimized CatBoost (Native Categorical Support)

To improve performance and reduce memory usage, we feed categorical features directly to CatBoost, allowing it to leverage internal target encoding.

In [ ]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import spearmanr
import gc
from catboost import CatBoostRegressor, Pool

print("Ready for optimization!")

Bereit für die Optimierung!


In [ ]:
print("Loading data...")
x_train_path = '/content/x_train_final_asAbTs5.csv'
y_train_path = '/content/y_train_final_YYyFil7.csv'

X_raw = pd.read_csv(x_train_path)
y_raw = pd.read_csv(y_train_path)

# Merge
id_col = 'Column1' if 'Column1' in X_raw.columns else X_raw.columns[0]
data = pd.merge(X_raw, y_raw, on=id_col, how='inner')

target_col = 'invalid_ratio'
y = data[target_col]
X = data.drop([target_col, id_col], axis=1)

del X_raw, y_raw
gc.collect()

# IMPORTANT: Define categorical columns
# CatBoost expects these as String or Integer, so we mark them explicitly.
# Fill NaNs, but DO NOT use get_dummies (native categorical support).
cat_features = ['day_of_week', 'month_of_year', 'hour']

# Ensure they are parsed as integers/strings (not floats)
for col in cat_features:
    X[col] = X[col].fillna(-1).astype(int).astype(str)

print("Data loaded. Categorical features marked:", cat_features)

Lade Daten...
Daten geladen. Kategorische Features markiert: ['day_of_week', 'month_of_year', 'hour']


In [ ]:
# Split Training/Validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training: {X_train.shape}, Validierung: {X_val.shape}")

Training: (4861236, 10), Validierung: (1215310, 10)


In [ ]:
print("Starting optimized training...")

model = CatBoostRegressor(
    iterations=2000,       # More iterations (was 500) for fine-tuning
    learning_rate=0.05,    # Slower learning rate = more precise results
    depth=10,              # Deeper trees understand complex patterns better
    l2_leaf_reg=5,         # L2 Regularization against overfitting
    loss_function='RMSE',
    eval_metric='RMSE',
    random_seed=42,
    task_type="CPU",       # Change to "GPU" if available in Colab for faster training
    verbose=100
)

# Pass 'cat_features' to the fit function! (Native Categorical)
model.fit(
    X_train, y_train,
    cat_features=cat_features, # This is the game changer
    eval_set=(X_val, y_val),
    early_stopping_rounds=100  # Early stopping if no improvement after 100 rounds
)

print("Training finished.")

Starte optimiertes Training...
0:	learn: 0.3657768	test: 0.3660292	best: 0.3660292 (0)	total: 9.75s	remaining: 5h 24m 57s
100:	learn: 0.3319217	test: 0.3319500	best: 0.3319500 (100)	total: 13m 24s	remaining: 4h 12m 5s
200:	learn: 0.3262671	test: 0.3263152	best: 0.3263152 (200)	total: 26m 36s	remaining: 3h 58m 10s
300:	learn: 0.3223504	test: 0.3224097	best: 0.3224097 (300)	total: 39m 43s	remaining: 3h 44m 11s
400:	learn: 0.3191875	test: 0.3192686	best: 0.3192686 (400)	total: 53m 11s	remaining: 3h 32m 6s
500:	learn: 0.3167095	test: 0.3168247	best: 0.3168247 (500)	total: 1h 6m 22s	remaining: 3h 18m 36s
600:	learn: 0.3147059	test: 0.3148522	best: 0.3148522 (600)	total: 1h 19m 28s	remaining: 3h 4m 59s
700:	learn: 0.3129053	test: 0.3130820	best: 0.3130820 (700)	total: 1h 32m 43s	remaining: 2h 51m 48s
800:	learn: 0.3113413	test: 0.3115455	best: 0.3115455 (800)	total: 1h 46m 18s	remaining: 2h 39m 7s
900:	learn: 0.3100271	test: 0.3102647	best: 0.3102647 (900)	total: 2h 1s	remaining: 2h 26m 24s


In [ ]:
print("Evaluating model...")
y_pred = model.predict(X_val)

spearman_score, _ = spearmanr(y_val, y_pred)
mse = mean_squared_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)

print("="*40)
print(f"🏆 Spearman Correlation: {spearman_score:.5f}")
print(f"📉 Mean Squared Error:   {mse:.5f}")
print(f"📊 R² Score:             {r2:.5f}")
print("="*40)

if spearman_score > 0.53:
    print("🚀 Better than the baseline model!")
else:
    print("Not yet beaten, maybe more iterations needed.")

Bewerte Modell...
🏆 Spearman Correlation: 0.55217
📉 Mean Squared Error:   0.09091
📊 R² Score:             0.33081
🚀 Besser als das Random Forest Modell!


In [ ]:
# --- SUBMISSION ---
print("Creating final file...")

# 1. Load test data
X_test_real = pd.read_csv('/content/x_test_final_fIrnA7Q.csv')

# 2. Prepare test data (Same logic: Drop ID, Categories as String)
if id_col in X_test_real.columns:
    X_test_real = X_test_real.drop(id_col, axis=1)
if target_col in X_test_real.columns:
    X_test_real = X_test_real.drop(target_col, axis=1)

for col in cat_features:
    X_test_real[col] = X_test_real[col].fillna(-1).astype(int).astype(str)

# 3. Retrain on ALL data?
# With early stopping, retraining without an eval_set is risky.
# TRICK: We use the already trained model (on 80%),
# since early stopping found the perfect convergence point.
# Retraining on 100% data without validation often leads to overfitting.
# So we use the trained model directly!

print("Generating predictions with the trained model...")
final_predictions = model.predict(X_test_real)

submission = pd.DataFrame(final_predictions, columns=['invalid_ratio'])
submission.index = np.arange(len(submission))
submission.index.name = None

filename = 'submission_catboost_optimized.csv'
submission.to_csv(filename, index=True)

print(f"Gespeichert als '{filename}'.")

from google.colab import files
try:
    files.download(filename)
except:
    pass

Erstelle finale Datei...
Erstelle Vorhersage mit dem trainierten Modell...
Gespeichert als 'submission_catboost_optimized.csv'.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 3. Advanced Neural Network with Spatial Embeddings

To match tree-based model performance on tabular data, this neural network uses **K-Means Clustering** to map GPS coordinates into distinct geographic zones. These zones, along with time features, are then learned via **Embedding Layers**.

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, callbacks
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import MiniBatchKMeans
from sklearn.model_selection import train_test_split
from scipy.stats import spearmanr
import gc

# Custom Callback: Calculates Spearman correlation live at the end of each epoch!
class SpearmanCallback(keras.callbacks.Callback):
    def __init__(self, validation_data):
        self.x_val = validation_data[0]
        self.y_val = validation_data[1]

    def on_epoch_end(self, epoch, logs={}):
        # Predict on validation data (verbose=0 to keep logs clean)
        y_pred = self.model.predict(self.x_val, batch_size=8192, verbose=0).flatten()
        score, _ = spearmanr(self.y_val, y_pred)
        print(f" — val_spearman: {score:.5f} 🏆")

print("Setup and Spearman tracking ready!")

Setup und Spearman-Tracking bereit!


In [ ]:
import pandas as pd
import numpy as np
import gc
from sklearn.cluster import MiniBatchKMeans
from sklearn.preprocessing import StandardScaler

print("Loading data...")
x_train_path = '/content/x_train_final_asAbTs5.csv'
y_train_path = '/content/y_train_final_YYyFil7.csv'

X_raw = pd.read_csv(x_train_path)
y_raw = pd.read_csv(y_train_path)

# Merge
id_col = 'Column1' if 'Column1' in X_raw.columns else X_raw.columns[0]
data = pd.merge(X_raw, y_raw, on=id_col, how='inner')

target_col = 'invalid_ratio'
y = data[target_col].astype('float32')
X = data.drop([target_col, id_col], axis=1)

del X_raw, y_raw
gc.collect()

# --- FIX: DETECT COLUMN NAMES ---
# Automatically find the correct coordinate columns
if 'longitude_scaled' in X.columns:
    lon_col, lat_col = 'longitude_scaled', 'latitude_scaled'
    print(f"Found scaled coordinates: {lon_col}, {lat_col}")
elif 'longitude' in X.columns:
    lon_col, lat_col = 'longitude', 'latitude'
    print(f"Found standard coordinates: {lon_col}, {lat_col}")
else:
    # Fallback: Guess column names based on substring 'lon'/'lat'
    lon_col = [c for c in X.columns if 'lon' in c.lower()][0]
    lat_col = [c for c in X.columns if 'lat' in c.lower()][0]
    print(f"Guessed coordinates: {lon_col}, {lat_col}")

# --- FEATURE ENGINEERING: SPATIAL CLUSTERING ---
print("Creating Geographic Zones (K-Means)...")
coords = X[[lon_col, lat_col]]

# Perform Spatial Clustering
kmeans = MiniBatchKMeans(n_clusters=300, random_state=42, batch_size=4096, n_init=3)
X['Zone_ID'] = kmeans.fit_predict(coords)

# Log-Transform for total_count to normalize skew
X['total_count'] = np.log1p(X['total_count'])

# Define features
cat_cols = ['day_of_week', 'month_of_year', 'hour', 'Zone_ID']
num_cols = ['total_count']

# Scale numerical features
scaler = StandardScaler()
X[num_cols] = scaler.fit_transform(X[num_cols]).astype('float32')

print("Data preparation and clustering successful!")

Lade Daten...
Habe skalierte Koordinaten gefunden: longitude_scaled, latitude_scaled
Erstelle Geografische Zonen (K-Means)...
Datenvorbereitung und Clustering erfolgreich!


In [ ]:
# Label Encoding for categories (so they start at 0, 1, 2...)
encoders = {}
for col in cat_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    encoders[col] = le # Save encoders for the test set

# Split Training / Validation
# Random split is sufficient for regression
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Create Input-Dictionaries for Keras Multi-Input model
def make_input_dict(df):
    inputs = {col: df[col].values for col in cat_cols}
    inputs['num_input'] = df[num_cols].values
    return inputs

train_inputs = make_input_dict(X_train)
val_inputs = make_input_dict(X_val)

print(f"Training Samples: {len(X_train)}")

Training Samples: 4861236


In [ ]:
# --- INPUTS ---
# Numerical Input
num_input = layers.Input(shape=(len(num_cols),), name='num_input')

# Categorical Inputs & Embeddings
cat_inputs = []
cat_embeddings = []

for col in cat_cols:
    # Input Layer
    inp = layers.Input(shape=(1,), name=col)
    cat_inputs.append(inp)

    # Embedding Dimension Rule of Thumb: min(50, vocab_size // 2)
    vocab_size = len(encoders[col].classes_)
    emb_dim = min(50, (vocab_size + 1) // 2)

    # Embedding Layer
    emb = layers.Embedding(input_dim=vocab_size + 1, output_dim=emb_dim)(inp)
    emb = layers.Flatten()(emb)
    cat_embeddings.append(emb)

# Concatenate all embeddings and numerical inputs
all_features = layers.Concatenate()([num_input] + cat_embeddings)

# --- DEEP LEARNING BLOCK ---
# Layer 1: Wide representation layer
x = layers.Dense(512, activation='relu')(all_features)
x = layers.BatchNormalization()(x) # Stabilizes training
x = layers.Dropout(0.3)(x)         # Prevents overfitting (Dropout)

# Layer 2: Feature compression
x = layers.Dense(256, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.3)(x)

# Layer 3: Final abstraction
x = layers.Dense(128, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.2)(x)

# Output Layer: Sigmoid activation for [0, 1] interval
output = layers.Dense(1, activation='sigmoid')(x)

# Build Model
model = keras.Model(inputs=[num_input] + cat_inputs, outputs=output)

# Optimizer: Adam with custom learning rate
optimizer = keras.optimizers.Adam(learning_rate=0.001)

# Compile
# We use MAE (Mean Absolute Error) as it is more robust to outliers than MSE
model.compile(optimizer=optimizer, loss='mae')

print("Model architecture ready!")
model.summary()

Modell Architektur steht!


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ day_of_week         │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ month_of_year       │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ hour (InputLayer)   │ (None, 1)         │          0 │ -                 │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Zone_ID             │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 1, 3)      │         21 │ day_of_week[0][0] │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 1, 6)      │         78 │ month_of_year[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_2         │ (None, 1, 7)      │        105 │ hour[0][0]        │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_3         │ (None, 1, 50)     │     15,050 │ Zone_ID[0][0]     │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ num_input           │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 3)         │          0 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_1 (Flatten) │ (None, 6)         │          0 │ embedding_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_2 (Flatten) │ (None, 7)         │          0 │ embedding_2[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten_3 (Flatten) │ (None, 50)        │          0 │ embedding_3[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 67)        │          0 │ num_input[0][0],  │
│ (Concatenate)       │                   │            │ flatten[0][0],    │
│                     │                   │            │ flatten_1[0][0],  │
│                     │                   │            │ flatten_2[0][0],  │
│                     │                   │            │ flatten_3[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 512)       │     34,816 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 512)       │      2,048 │ dense[0][0]       │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 512)       │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 256)       │    131,328 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 218,007 (851.59 KB)

 Trainable params: 216,215 (844.59 KB)

 Non-trainable params: 1,792 (7.00 KB)

In [ ]:
print("Starting Training...")

# Callbacks
# 1. Live Spearman Tracker
spearman_cb = SpearmanCallback((val_inputs, y_val.values))

# 2. Reduce learning rate on plateau
reduce_lr = callbacks.ReduceLROnPlateau(
    monitor='val_loss', factor=0.5, patience=3, min_lr=1e-5, verbose=1
)

# 3. Early Stopping to save time and prevent overfitting
early_stop = callbacks.EarlyStopping(
    monitor='val_loss', patience=8, restore_best_weights=True, verbose=1
)

# Training
history = model.fit(
    train_inputs, y_train,
    validation_data=(val_inputs, y_val),
    epochs=50,             # Max 50 epochs
    batch_size=4096,       # Large batch size for speed in Colab
    callbacks=[spearman_cb, reduce_lr, early_stop],
    verbose=1
)

print("Training finished.")

Starte Training...
Epoch 1/50
1187/1187 ━━━━━━━━━━━━━━━━━━━━ 0s 204ms/step - loss: 0.2558 — val_spearman: 0.40391 🏆
1187/1187 ━━━━━━━━━━━━━━━━━━━━ 280s 231ms/step - loss: 0.2558 - val_loss: 0.2383 - learning_rate: 0.0010
Epoch 2/50
1187/1187 ━━━━━━━━━━━━━━━━━━━━ 0s 200ms/step - loss: 0.2395 — val_spearman: 0.40327 🏆
1187/1187 ━━━━━━━━━━━━━━━━━━━━ 261s 220ms/step - loss: 0.2395 - val_loss: 0.2373 - learning_rate: 0.0010
Epoch 3/50
1187/1187 ━━━━━━━━━━━━━━━━━━━━ 0s 203ms/step - loss: 0.2384 — val_spearman: 0.40358 🏆
1187/1187 ━━━━━━━━━━━━━━━━━━━━ 274s 230ms/step - loss: 0.2384 - val_loss: 0.2369 - learning_rate: 0.0010
Epoch 4/50
1187/1187 ━━━━━━━━━━━━━━━━━━━━ 0s 206ms/step - loss: 0.2379 — val_spearman: 0.40519 🏆
1187/1187 ━━━━━━━━━━━━━━━━━━━━ 316s 225ms/step - loss: 0.2379 - val_loss: 0.2365 - learning_rate: 0.0010
Epoch 5/50
1187/1187 ━━━━━━━━━━━━━━━━━━━━ 0s 209ms/step - loss: 0.2378 — val_spearman: 0.40484 🏆
1187/1187 ━━━━━━━━━━━━━━━━━━━━ 281s 237ms/step - loss: 0.2378 - val_loss: 0.

In [ ]:
print("Creating submission...")

# 1. Load test data
X_test_real = pd.read_csv('/content/x_test_final_fIrnA7Q.csv')

# Save ID for later checks, but remove from feature set X
if id_col in X_test_real.columns:
    X_test_real = X_test_real.drop(id_col, axis=1)
if target_col in X_test_real.columns:
    X_test_real = X_test_real.drop(target_col, axis=1)

# 2. Apply Zone-Clustering on test data (IMPORTANT!)
# FIX: We use the automatically found column names from Block 2
print(f"Using coordinates for test-clustering: {lon_col}, {lat_col}")
coords_test = X_test_real[[lon_col, lat_col]]
X_test_real['Zone_ID'] = kmeans.predict(coords_test)

# 3. Log-Transform
X_test_real['total_count'] = np.log1p(X_test_real['total_count'])

# 4. Numerical Scaling
X_test_real[num_cols] = scaler.transform(X_test_real[num_cols]).astype('float32')

# 5. Category Encoding (Handling unknown labels)
for col in cat_cols:
    # We map using the saved encoder dictionaries
    le = encoders[col]
    # Create a mapping: Label -> Integer
    mapping = dict(zip(le.classes_, le.transform(le.classes_)))

    # Apply mapping and fill unknown values with 0
    X_test_real[col] = X_test_real[col].map(mapping).fillna(0).astype(int)

# 6. Input Dict for Keras
test_inputs = make_input_dict(X_test_real)

# 7. Prediction
print("Calculating predictions...")
pred_nn = model.predict(test_inputs, batch_size=8192, verbose=1).flatten()

# 8. Save
submission = pd.DataFrame(pred_nn, columns=['invalid_ratio'])
submission.index = np.arange(len(submission))
submission.index.name = None
filename = 'submission_nn_optimized.csv'
submission.to_csv(filename, index=True)

print(f"Done! File '{filename}' saved.")

# Download Button (Colab)
from google.colab import files
try:
    files.download(filename)
except:
    pass

Erstelle Submission...
Nutze Koordinaten für Test-Clustering: longitude_scaled, latitude_scaled
Berechne Vorhersagen...
248/248 ━━━━━━━━━━━━━━━━━━━━ 23s 92ms/step
Fertig! Datei 'submission_nn_optimized.csv' wurde gespeichert.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>